### <b> Code used for the first revision

In [1]:
import os
import open3d as o3d
import numpy as np
from matplotlib import pyplot as plt
import meshplot as mp
from Display import Colors
from Flatten import write_TLC_input, read_TLC_result
from FeatureLines import get_mesh, to_pseudo_PLY2, ReadCrestLine

In [ ]:
neighbor = 4
mesh_name = f'name of mesh (without file extension)' 
# NOTE: exported with forward_axis=Y, up_axis=Z
here = "file path of the mesh"
mesh_file = f"{mesh_name}.ply"
ply2_name = f"{mesh_name}_k={neighbor}_ply2.txt"
os.chdir(here)
V, F = get_mesh(mesh_file)


In [ ]:
"""Crest Lines"""
os.chdir(here)
print(f"3D Lidar data: \t{mesh_file}")
print(f"neighbor value: k = {neighbor}")
print(f"PLY2 file: \t{ply2_name}")
to_pseudo_PLY2(name=ply2_name, verts=V, faces=F, neighbor=neighbor)
crestcode_input = f"./setCurvature {ply2_name} output.txt"
print(f"in CrestCODE, enter: {crestcode_input}")
keeper = "ravines.txt"
os.rename(ply2_name, f'file path to CrestCODE/{ply2_name}')
os.chdir('file path to ')
os.system('pwd')
os.system(crestcode_input)
os.system(f'rm {ply2_name}')
os.system('pwd')
os.rename('ravines.txt', f'file path to this folder/{mesh_name}_k={neighbor}_ravines.txt')
os.rename('ridges.txt', f'file path to this folder/{mesh_name}_k={neighbor}_ridges.txt')
os.chdir(here)
os.system('pwd')
crestline_V_3d, crestline_E = ReadCrestLine(f"{mesh_name}_k={neighbor}_ravines.txt")

In [ ]:
from scipy import spatial
landmarks_mesh_OG = np.loadtxt('mesh landmarks file', delimiter=',')
print(landmarks_mesh_OG)
sorted_indices = landmarks_mesh_OG[:, 0].argsort()
landmarks_mesh_OG = landmarks_mesh_OG[sorted_indices]
landmarks_mesh_OG = landmarks_mesh_OG[:, 1:4]
landmarks_mesh_OG[:, 2] = 0
print(landmarks_mesh_OG)
landmarks_mesh = np.zeros(shape=landmarks_mesh_OG.shape)
landmarks_mesh_indices = []
V_orthographic_project_z = V.copy()
V_orthographic_project_z[:, 2] = 0
tree = spatial.KDTree(V_orthographic_project_z)
for i, landmark in enumerate(landmarks_mesh_OG):
    distance, index = tree.query(landmark)
    landmarks_mesh[i] = V_orthographic_project_z[index]  
    landmarks_mesh_indices.append(index)
print(landmarks_mesh)  

# new landmarks: landmarks_gpr_locations.txt
# old landmarks: old_landmarks_GPR_locations
landmarks_gpr = np.loadtxt('gpr landmarks file', delimiter=',')
print(landmarks_gpr)
sorted_indices = landmarks_gpr[:, 0].argsort()
landmarks_gpr = landmarks_gpr[sorted_indices]
landmarks_gpr = landmarks_gpr[:, 1:4]
landmarks_gpr[:, 2] = 0
print(landmarks_gpr)

np.savetxt(X=landmarks_mesh, fname='landmarks_mesh.txt')

In [ ]:
"""ARAP deform in preparation of SEA algorithm"""
# load handle information
PALETTE = Colors()
# This function beaten by octree query (scipy)
# def closest_point(locations : NDArray, points : NDArray):
#     def closest_location_index(location : NDArray):
#         displacement = points - location
#         distance = np.einsum('ij,ij->i', displacement, displacement)
#         return np.argmin(distance)
#     closest_indices = []
#     for location in locations:
#         closest_indices.append(closest_location_index(location))
#     return closest_indices
grey = np.array([0.99, 0.99, 0.99]) # grey
mesh_color = np.zeros(shape=V.shape)
mesh_color[:, :] = PALETTE.GREY
mesh_color[0] = PALETTE.TURQUOISE
target_pos = landmarks_gpr
handle_pos = landmarks_mesh
arap_before = mp.plot(v=V, f=F, c=mesh_color)
arap_before.add_points(points=landmarks_mesh, c='red',
                          shading={"point_size": 0.1})
arap_before.add_points(points=landmarks_gpr, c='blue',
                          shading={"point_size": 0.1})
# arap_before.add_lines(beginning=landmarks_mesh, ending=landmarks_mesh, 
#                       shading={"line_color": "blue", "line_width": 1000.0})
# arap_before.add_lines(beginning=crestline_V_3d[crestline_E[:,0]], 
#                   ending=crestline_V_3d[crestline_E[:,1]], 
#                   shading={"line_color": "black", "line_width": 1000.0})

In [ ]:
"""2D visual on landmarks"""
n = np.arange(landmarks_mesh.shape[0])

fig, ax = plt.subplots()
ax.scatter(landmarks_mesh[:, 0], landmarks_mesh[:, 1], c='blue')
ax.scatter(landmarks_gpr[:, 0], landmarks_gpr[:, 1], c='red')

for i, txt in enumerate(n):
    ax.annotate(txt, (landmarks_mesh[:, 0][i], landmarks_mesh[:, 1][i]))
    ax.annotate(txt, (landmarks_gpr[:, 0][i], landmarks_gpr[:, 1][i]))

In [8]:
"""Export Crest Lines (3D file)"""
os.chdir(here)
def export_lines(vertices, edges, filename):
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(vertices),
        lines=o3d.utility.Vector2iVector(edges),
    )
    o3d.io.write_line_set(filename, line_set, print_progress=True)

export_lines(vertices=crestline_V_3d,
             edges=crestline_E[:,0:2].astype(int),
             filename=f"{mesh_name}_n={neighbor}_3d_lines.ply")
print(f"saved mesh file: {mesh_name}_n={neighbor}_3d_lines.ply")

Writing PLY: [========================================] 100%
saved mesh file: tbk_d=11_USE_n=4_3d_lines.ply


In [ ]:
"""ARAP (with libigl)"""
import igl
handle_ids_b = np.asarray([[id] for id in landmarks_mesh_indices])
arap_object = igl.ARAP(V, F, 3, handle_ids_b)
V_arap_igl = arap_object.solve(landmarks_gpr, V)
# libigl runs faster (after adding libigl's precomputation time)
arap_result = mp.plot(v=V_arap_igl, f=F, c=PALETTE.GREEN)

In [ ]:
"""SEA (rubbersheet) to rubber-sheet crestlines to fit GPR data!"""
TLC_filename = f"{mesh_name}_tlc.txt"
flat_filename = f"{mesh_name}_2d.txt"
write_TLC_input(
    filename=TLC_filename,
    restVert=V,
    initVert=V_arap_igl[:,0:2],
    triangles=F,
    handles=np.asarray(landmarks_mesh_indices))
print(f"the TLC file is: {TLC_filename}")
SEA_input = f"./SEA_QN {TLC_filename} solver_options.txt {flat_filename}"
print(f"in SEA, enter: {SEA_input}")
os.chdir(here)
os.rename(TLC_filename, f"file path to SEA/cmake-build-debug/{TLC_filename}")
os.chdir('file path to SEA/cmake-build-debug')
os.system('pwd')
os.system(SEA_input)
os.rename(flat_filename, f'file path to this folder/{flat_filename}')
os.system(f'rm {TLC_filename}')

In [134]:
"""mesh after SEA algorithm"""
os.chdir(here)
dictionary = read_TLC_result(flat_filename)
V_2d = np.zeros(V.shape)
V_2d[:,0:2] = dictionary["resV"]
# flat_plot = mp.plot(v=V_2d, f=F, c=mesh_color)
# flat_plot.add_points(
#   points=V_2d[landmarks_mesh_indices], c='blue', shading={"point_size": 2})
# mesh_lines(vertices=V_2d, faces=F, plot=flat_plot, color="black")

In [ ]:
"""Rubbersheet Crest Lines by proxy of mesh"""
print(crestline_V_3d.shape)
import importlib as imp
import Transfer
imp.reload(Transfer)
mover = Transfer.crestline_mover( 
            mesh_3d_vertices=V, mesh_faces=F, 
            mesh_3d_crestlines_vertices=crestline_V_3d, 
            crestline_edges=crestline_E, 
            mesh_2d_vertices=V_2d, color=mesh_color)
"""display crestlines (skipping the special cases, where one or more of the 
end points do NOT fall on the edge of a mesh, see CrestCODE paper!)"""
# use valid_beginning & valid_ending below for crestline vertices, 
# NOT mover.mesh_2d_crestlines_vertices!
valid_beginning = []
valid_ending = []
tolerance = 1e-3 # if distance to origin is less than this, the point is omitted
valid_crest_line_edges_indices_B = []
valid_crest_line_edges_indices_E = []
for u, v, _ in crestline_E:
    b = mover.mesh_2d_crestlines_vertices[u]
    e = mover.mesh_2d_crestlines_vertices[v]
    if np.dot(b, b) <= tolerance or np.dot(b, b) <= tolerance:
        continue
    else:
        valid_beginning.append(b)
        valid_ending.append(e)
        valid_crest_line_edges_indices_B.append(u)
        valid_crest_line_edges_indices_E.append(v)
    # print(f"beginning: {beginning}, ending: {ending}")
    # break
flat_plot_lines = mp.plot(v=V_2d, f=F, c=mesh_color)
flat_plot_lines.add_lines(
                  beginning=np.asarray(valid_beginning), 
                  ending=np.asarray(valid_ending), 
                  shading={"line_color": "red", "line_width": 1000.0})
# mesh_lines(vertices=V_2d, faces=F, plot=flat_plot_lines, color="black")
flat_plot_lines.add_points(
  points=V_2d[landmarks_mesh_indices] + [0, 0, 0.005], # <- big points: where the small points WILL BE
  c='green',
  shading={"point_size": 2})


In [97]:
"""save mesh & deformed mesh data"""
np.savetxt(fname=f'{mesh_name}_V.txt', X=V, fmt='%1.17f')
np.savetxt(fname=f'{mesh_name}_F.txt', X=V_2d, fmt='%1.17f')
# np.savetxt(fname=f'{mesh_name}_crest-line_3d_V.txt', X=crestline_V_3d, fmt='%1.17f')
np.savetxt(fname=f'{mesh_name}_crest-line_3d_B.txt', X=mover.mesh_3d_crestlines_vertices[valid_crest_line_edges_indices_B], fmt='%1.17f')
np.savetxt(fname=f'{mesh_name}_crest-line_3d_E.txt', X=mover.mesh_3d_crestlines_vertices[valid_crest_line_edges_indices_E], fmt='%1.17f')
# save crestline 2d vertices as their beginnings and endings
np.savetxt(fname=f'{mesh_name}_crest-line_2d_B.txt', X=np.asarray(valid_beginning), fmt='%1.17f')
np.savetxt(fname=f'{mesh_name}_crest-line_2d_E.txt', X=np.asarray(valid_ending), fmt='%1.17f')
# np.savetxt(fname=f'{mesh_name}_crest-line_E.txt', X=crestline_E, fmt='%i')
np.savetxt(fname=f'{mesh_name}_F.txt', X=F, fmt='%i')

In [14]:
from PIL import Image, ImageOps
from Display import Plot_2D
from DualColor import binary

In [15]:
img_display = Image.open('background image')
img_display = np.flip(img_display, axis=0)
img_display.shape

(4168, 3184, 4)

In [ ]:
"""plot mesh mask"""
crestline_area_mask_image_filename = f'{mesh_name}_mesh_mask.png'
if os.path.isfile(crestline_area_mask_image_filename):
    print(f"loading existing image")
    # load image, turn into path
    crestlines_mask_image = Image.open(crestline_area_mask_image_filename)
    crestlines_mask, _ = binary(np.asarray(ImageOps.grayscale(crestlines_mask_image)), 0, [255, 255, 255], [0, 0, 0])
    # print(crestlines_mask.shape)
    # plt.imshow(crestlines_mask, cmap='gray')
else:
    print(f"making new image")
    img_display = Image.open('background image')
    # (255 - np.asarray(img_grayscale)) # <- numpy alternative
    img_display = np.flip(img_display, axis=0)
    y_length, x_length, _ = img_display.shape
    # use plotter object!
    thic = 1.6
    thin = 0.1
    plotter = Plot_2D(init_color='black', warp_color='white', thickness=thin)

    f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
    print(f"image is {x_length}, {y_length}")
    ax.set_xlim(0, x_length)
    ax.set_ylim(0, y_length)

    plotter.Plot_Background(f, ax, img_display, "BLACK")
    for face in F:
        v1, v2, v3 = face
        triangle = plt.Polygon((V_2d[v1][0:2] * 1000, 
                                V_2d[v2][0:2] * 1000, 
                                V_2d[v3][0:2] * 1000), 
            facecolor='white', edgecolor='white')
        ax.add_patch(p=triangle)

    ax.autoscale()
    current_figure = plt.gcf()
    plt.show()
    plt.draw()
    """save part of figure"""
    DPI = 169.18 # manually tuned
    plotter.Plot_SaveTrim(f, ax, current_figure, crestline_area_mask_image_filename, 
                        (0, 0), (x_length, x_length)) # save all
# load image, turn into mask
crestlines_mask_image = Image.open(crestline_area_mask_image_filename)
crestlines_mask, _ = binary(np.asarray(ImageOps.grayscale(crestlines_mask_image)), 0, [255, 255, 255], [0, 0, 0])
np.savetxt(fname=f'{mesh_name}_mesh_mask.txt', X=crestlines_mask, fmt='%i')

In [ ]:
"""plot GPR mask"""
crestline_area_mask_image_filename = f'{mesh_name}_gpr_mask.png'
if os.path.isfile(crestline_area_mask_image_filename):
    print("reading from existing image")
else:
    print(f"making new image")
    radar_image = Image.open('background image')
    # (255 - np.asarray(img_grayscale)) # <- numpy alternative
    img_display = np.flip(radar_image, axis=0)
    y_length, x_length, _ = img_display.shape
    radar_mask_upright = np.zeros(shape=(y_length, x_length))
    for x in range(x_length):
        for y in range(y_length):
            if img_display[y][x][3] == 0:
                radar_mask_upright[y][x] = 1
    # plt.imshow(radar_mask_upright)
    # use plotter object!
    thic = 1.6
    thin = 0.1
    plotter = Plot_2D(init_color='black', warp_color='white', thickness=thin)

    f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
    print(f"image is {x_length}, {y_length}")
    ax.set_xlim(0, x_length)
    ax.set_ylim(0, y_length)

    # plt.imshow(img_display)
    plotter.Plot_Background(f, ax, img_display, "BLACK")
    plotter.Plot_ImageTransform(f, ax, 1 - (radar_mask_upright) * 255)
    # plotter.Plot_Handle(f, ax, handle_dictionary, V)
    # plotter.Plot_Crestlines_Init(f, ax, crestline_V_3d, mover.mesh_2d_crestlines_vertices, crestline_E)
    # plotter.Plot_Crestlines_Warp(f, ax, crestline_V_3d, crestline_E)
    # plotter.Plot_Labels(f, ax)

    ax.autoscale()
    current_figure = plt.gcf()
    plt.show()
    plt.draw()
    """save part of figure"""
    DPI = 169.18 # manually tuned
    plotter.Plot_SaveTrim(f, ax, current_figure, crestline_area_mask_image_filename, 
                        (0, 0), (x_length, x_length)) # save all
# get GPR image area mask & union mask
radar_mask_image = Image.open(crestline_area_mask_image_filename)
radar_mask, _ = binary(np.asarray(ImageOps.grayscale(radar_mask_image)), 0, [255, 255, 255], [0, 0, 0])
np.savetxt(fname=f'{mesh_name}_gpr_mask.txt', X=radar_mask, fmt='%i')

reading from existing image


In [ ]:
"""calculate the area in which comparison is done, represented by mask"""
union_mask = crestlines_mask * radar_mask
union_mask = np.asarray(union_mask)
np.savetxt(fname=f'{mesh_name}_union_mask.txt', X=union_mask, fmt='%i')

In [ ]:
"""plot Crest Lines (POST-rubbersheet) mask"""
import Display
imp.reload(Display)
filename = f'{mesh_name}_crest-lines-2d_mask.png'
if os.path.isfile(filename):
    print("use existing image")
else:
    print("making new image")
    radar_image = Image.open('background image')
    # (255 - np.asarray(img_grayscale)) # <- numpy alternative
    img_display = np.flip(radar_image, axis=0)
    y_length, x_length, _ = img_display.shape

    plotter = Display.Plot_2D(init_color='black', warp_color='white', thickness=0.1)
    f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
    # print(f"image is {self.x_length}, {self.y_length}")
    ax.set_xlim(0, x_length)
    ax.set_ylim(0, y_length)
    plotter.Plot_Background(f, ax, img_display, "black")
    plotter.Plot_Crestlines_Warp(f, ax, begin=valid_beginning, end=valid_ending)
    ax.autoscale()
    current_figure = plt.gcf()
    plt.show()
    plt.draw()
    plotter.Plot_SaveTrim(f, ax, current_figure, filename, 
                        (0, 0), (x_length, x_length)) # save all
print(filename)
crestlines_lines_mask_image = Image.open(filename)
crestlines_lines_mask, _ = binary(np.asarray(ImageOps.grayscale(crestlines_lines_mask_image)), 0, [255, 255, 255], [0, 0, 0])
np.savetxt(fname=f'{mesh_name}_crest-lines-2d_mask.txt', X=crestlines_lines_mask, fmt='%i')
a = crestlines_lines_mask.copy()

In [ ]:
"""plot Crest Lines (Original) mask"""
import Display
imp.reload(Display)
filename = f'{mesh_name}_crest-lines-3d_mask.png'
if os.path.isfile(filename):
    print("use existing image")
else:
    print("making new image")
    radar_image = Image.open('background image')
    # (255 - np.asarray(img_grayscale)) # <- numpy alternative
    img_display = np.flip(radar_image, axis=0)
    y_length, x_length, _ = img_display.shape

    plotter = Display.Plot_2D(init_color='black', warp_color='white', thickness=0.1)
    f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
    # print(f"image is {self.x_length}, {self.y_length}")
    ax.set_xlim(0, x_length)
    ax.set_ylim(0, y_length)
    plotter.Plot_Background(f, ax, img_display, "black")
    plotter.Plot_Crestlines_Warp(f, ax, 
        begin=mover.mesh_3d_crestlines_vertices[valid_crest_line_edges_indices_B], 
        end=mover.mesh_3d_crestlines_vertices[valid_crest_line_edges_indices_E])
    ax.autoscale()
    current_figure = plt.gcf()
    plt.show()
    plt.draw()
    plotter.Plot_SaveTrim(f, ax, current_figure, filename, 
                        (0, 0), (x_length, x_length)) # save all
print(filename)
crestlines_lines_mask_image = Image.open(filename)
crestlines_lines_mask, _ = binary(np.asarray(ImageOps.grayscale(crestlines_lines_mask_image)), 0, [255, 255, 255], [0, 0, 0])
np.savetxt(fname=f'{mesh_name}_crest-lines-3d_mask.txt', X=crestlines_lines_mask, fmt='%i')
b = crestlines_lines_mask.copy()

In [ ]:
"""VISUAL: Crest Lines & Rubbersheet before-and-after"""
imp.reload(Display)
from matplotlib import collections as mc
conversion=1000

# handle_indices = [19841, 12438, 26200, 31800, 33350, 78265, 81096, 21838, 68046, 27896, 58194, 11679, 44274, 21332, 97337, 78209]
# handle_positions = mesh_3d_vertices[handle_indices]
# handle_positions = np.asarray([pos[:2] * conversion for pos in handle_positions])

# GPR image: 25~89.png
# white image: white.png
img_display = np.asarray(Image.open('25~89.png').convert('L'))
img_display = np.flip(img_display, axis=0)
x_length, y_length = img_display.shape
# plotter = Display.Plot_2D(init_color='black', warp_color='white', thickness=1)
f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
ax.set_xlim(0, x_length)
ax.set_ylim(0, y_length)
    # plt.imshow(img_display)
# plotter.Plot_Background(f, ax, img_display, "BLACK")
"""GPR image"""
import matplotlib as mpl
img_display.fill(255) # make all white
# print(img_display[0, 0])
y_length, x_length = img_display.shape
imgplot = ax.imshow(img_display, cmap=plt.cm.gray, vmin=0, vmax=255)
transform = mpl.transforms.Affine2D().translate(-x_length/2, -y_length/2)
transform = transform.scale(0.855) # need conversion
transform = transform.rotate(np.deg2rad(-17.7447))
transform = transform.translate(1470, 1260) # same amount as in blender
imgplot.set_transform(transform + ax.transData)

# ax.plot(1000, 1000, 'o', markersize=12, color='green')
"""add 3d crest lines"""
lines = [] # item = [(start coord tuple), (end coord tuple)]
for u, v in zip(valid_crest_line_edges_indices_B, valid_crest_line_edges_indices_E):
    x1, y1, _ = mover.mesh_3d_crestlines_vertices[u] * conversion
    x2, y2, _ = mover.mesh_3d_crestlines_vertices[v] * conversion
    lines.append([(x1, y1), (x2, y2)])
    # print([(x1, y1), (x2, y2)])
    # break
crest_line_3d_collection = mc.LineCollection(lines, colors='blue', linewidths=0.1, linestyle='solid')
ax.add_collection(crest_line_3d_collection)
"""add 2d (rubbersheeted) crest lines"""
# lines = [] # item = [(start coord tuple), (end coord tuple)]
# for u, v in zip(valid_beginning, valid_ending):
#     x1, y1, _ = u * conversion
#     x2, y2, _ = v * conversion
#     lines.append([(x1, y1), (x2, y2)])
#     # print([(x1, y1), (x2, y2)])
#     # break
# crest_line_2d_collection = mc.LineCollection(lines, colors='red', linewidths=0.1, linestyle='solid')
# ax.add_collection(crest_line_2d_collection)
"""add landmarks"""
ax.scatter(x=landmarks_gpr[:,0] * conversion, 
           y=landmarks_gpr[:,1] * conversion, 
    marker="D", s=0.5, c='red')
ax.scatter(x=landmarks_mesh[:,0] * conversion, 
           y=landmarks_mesh[:,1] * conversion, 
    marker="o", s=0.5, c='blue')
"""GPR landmarks, but in mesh landmarks style"""
# ax.scatter(x=landmarks_gpr[:,0] * conversion, 
#            y=landmarks_gpr[:,1] * conversion, 
#     marker="o", s=0.5, c='blue')
"""add mesh edges (Rubbersheet)"""
# lines = []
# for u, v in zip(F[:, 0], F[:, 1]):
#     x1, y1, _ = mover.mesh_2d_vertices[u] * conversion
#     x2, y2, _ = mover.mesh_2d_vertices[v] * conversion
#     lines.append([(x1, y1), (x2, y2)])
# for u, v in zip(F[:, 0], F[:, 2]):
#     x1, y1, _ = mover.mesh_2d_vertices[u] * conversion
#     x2, y2, _ = mover.mesh_2d_vertices[v] * conversion
#     lines.append([(x1, y1), (x2, y2)])
# for u, v in zip(F[:, 1], F[:, 2]):
#     x1, y1, _ = mover.mesh_2d_vertices[u] * conversion
#     x2, y2, _ = mover.mesh_2d_vertices[v] * conversion
#     lines.append([(x1, y1), (x2, y2)])
# mesh_lines_collection = mc.LineCollection(lines, colors='green', linewidths=0.05, linestyle='solid')
# ax.add_collection(mesh_lines_collection)
"""add mesh edges original"""
lines = []
for u, v in zip(F[:, 0], F[:, 1]):
    x1, y1, _ = V[u] * conversion
    x2, y2, _ = V[v] * conversion
    lines.append([(x1, y1), (x2, y2)])
for u, v in zip(F[:, 0], F[:, 2]):
    x1, y1, _ = V[u] * conversion
    x2, y2, _ = V[v] * conversion
    lines.append([(x1, y1), (x2, y2)])
for u, v in zip(F[:, 1], F[:, 2]):
    x1, y1, _ = V[u] * conversion
    x2, y2, _ = V[v] * conversion
    lines.append([(x1, y1), (x2, y2)])
mesh_lines_collection = mc.LineCollection(lines, colors='green', linewidths=0.05, linestyle='solid')
ax.add_collection(mesh_lines_collection)

ax.autoscale()
ax.set_axis_off()
current_figure = plt.gcf()
plt.show()
plt.draw()
# DPI = 169.18 # for exact same number of pixels as input
DPI = 1280
needs_mask = "mesh-lines-before"
#   "crest-lines_before-and-after-deform.png"

"""Crop"""
# print(DPI)
# from matplotlib.transforms import Bbox
# lower_left = (1300, 2300)
# lower_right = (1500, 2500)
# x0, y0 = lower_left
# x1, y1 = lower_right
# bbox = Bbox([[x0, y0],[x1, y1]])
# bbox = bbox.transformed(ax.transData).transformed(f.dpi_scale_trans.inverted())
# current_figure.savefig(filename, dpi=DPI, bbox_inches=bbox, pad_inches=0, facecolor=f.get_facecolor())

"""Save full image"""
current_figure.savefig(f"{needs_mask}.png", dpi=DPI,
    bbox_inches='tight', pad_inches=0)

"""cropping"""
# needs_mask_image = np.asarray(Image.open(f"{needs_mask}.png").convert('RGB'))
# print(needs_mask_image.shape)
# mask = np.loadtxt('union_mask.txt').astype(int)
# # plt.imshow(mask)
# needs_mask_image = needs_mask_image * np.repeat(mask[:, :, None], repeats=3, axis=2)
# print(needs_mask_image.shape)
# # plt.imshow(needs_mask_image, alpha=mask)

# print(needs_mask_image.shape, needs_mask_image.max())
# mask = mask[:, :, None] * 255
# needs_mask_image = np.concatenate([needs_mask_image, mask], axis=2)
# print(needs_mask_image.shape)
# print(needs_mask_image[1000, 1000])
# needs_mask_image = needs_mask_image.astype(np.uint8)
# square_side_length = needs_mask_image.shape[1]
# print(needs_mask_image.shape)
# final = Image.fromarray(needs_mask_image, mode="RGBA")
# final.save(f"{needs_mask}_masked.png")

In [ ]:
DPI = 1280
needs_mask = "landmarks_and_mesh-edges"
"""Save full image"""
current_figure.savefig(f"{needs_mask}.png", dpi=DPI,
    bbox_inches='tight', pad_inches=0)

In [ ]:
import Radar_drawn
background = np.asarray(Image.open('background image')) # square
print(f"background shape: {np.asarray(background).shape}")
image_mask = np.loadtxt(f'union_mask.txt') # not square
mask_x, mask_y = image_mask.shape
image_mask = image_mask[mask_x - mask_y:]
print(f"mask shape: {image_mask.shape}")
# culled_background = background * image_mask

f, ax = plt.subplots(figsize=(16, 8), ncols=2)
culled_background = background[:, :, :3] * image_mask[:, :, np.newaxis]
culled_background_flip = 255 - culled_background
ax[0].imshow((culled_background).astype(np.uint8))
ax[1].imshow((culled_background_flip).astype(np.uint8))

crest_lines_2d_mask = np.loadtxt(f'{mesh_name}_crest-lines-2d_mask.txt')
crest_lines_3d_mask = np.loadtxt(f'{mesh_name}_crest-lines-3d_mask.txt')
image_lines_composite_mask = np.loadtxt("image_ridges_composite.txt computed by 2_TBK_GPR.ipynb")
mask = np.loadtxt(f'{mesh_name}_union_mask.txt') # NOTE: same as image_mask
# apply the mask, so we only compare what's inside the area it represents
crest_lines_2d_mask = crest_lines_2d_mask * mask
crest_lines_3d_mask = crest_lines_3d_mask * mask
radar_mask = image_lines_composite_mask * mask

In [ ]:
"""Utility: scale conversion & color conversion"""
def irl_to_pixel(irl_length : float):
    return int(irl_length 
        * (x_upper_bound - x_lower_bound) / (204.597716)
        )
test_irl_radius = 0.8
print(f"{test_irl_radius} is {irl_to_pixel(test_irl_radius)} pixels")
h = "#32a852"
def toRGB(h : str):
    return np.asarray(list(int(h.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)))
toRGB(h)

In [ ]:
filename = 'OLD_CREST-LINES' + '.png'
print(filename)
crestlines_lines_mask_image = Image.open(filename)
mask, _ = binary(
    np.asarray(ImageOps.grayscale(crestlines_lines_mask_image)), 
    0, [255, 255, 255], [0, 0, 0]) 

In [123]:
mask = np.loadtxt(f'{mesh_name}_crest-lines-2d_mask.txt')

In [ ]:
"""Estimate scale w/ bounding box & rubbersheet Crest Lines"""
crest_lines_2d_B = np.loadtxt(f'{mesh_name}_crest-line_2d_B.txt')
crest_lines_2d_E = np.loadtxt(f'{mesh_name}_crest-line_2d_E.txt')

pc_max_x = max(crest_lines_2d_B[:, 0].max(), crest_lines_2d_E[:, 0].max())
pc_max_y = max(crest_lines_2d_B[:, 1].max(), crest_lines_2d_E[:, 1].max())
pc_min_x = min(crest_lines_2d_B[:, 0].min(), crest_lines_2d_E[:, 0].min())
pc_min_y = min(crest_lines_2d_B[:, 1].min(), crest_lines_2d_E[:, 1].min())
print(f"bounding box in Point Cloud units: "
      f"\n\tx in {pc_min_x, pc_max_x},"
      f"\n\ty in {pc_min_y, pc_max_y}")
image_x, image_y = mask.shape
x_lower_bound = 0
for x in range(image_x):
    if np.sum(mask[x]) == 0:
        x_lower_bound = x
    else:
        break
x_upper_bound = image_x
for x in range(image_x).__reversed__():
    if np.sum(mask[x]) == 0:
        x_upper_bound = x
    else:
        break
y_lower_bound = 0
for y in range(image_y):
    if np.sum(mask[:,y]) == 0:
        y_lower_bound = y
    else:
        break
y_upper_bound = image_y
for y in range(image_y).__reversed__():
    if np.sum(mask[:,y]) == 0:
        y_upper_bound = y
    else:
        break
print(f"bounding box in pixels: x in {x_lower_bound, x_upper_bound}")
print(f"bounding box in pixels: y in {y_lower_bound, y_upper_bound}")
thicken = 40
x_hi = x_upper_bound + thicken
x_lo = x_lower_bound - thicken
y_hi = y_upper_bound + thicken
y_lo = y_lower_bound - thicken
print(f"displaying cropper box in pixels: x in {x_lo, x_hi}")
print(f"displaying cropper box in pixels: y in {y_lo, y_hi}")

In [ ]:
"""Visual: confusion matrix"""
imp.reload(Radar_drawn)
crest_lines_to_use = crest_lines_3d_mask # mask 
# 1.2=11, 1.1=10, 1.0=9, 0.9=8, 0.8=7 <- computed from above utilities
irl_radius = 1.0 # current: 1.1
pixel_radius = irl_to_pixel(irl_radius)
# Optional: make folder
folder = ""# f"irl={irl_radius}_px={pixel_radius}"
# os.mkdir(folder)
print(f"{irl_radius} in real life is {pixel_radius} in the picture")
comparer = Radar_drawn.Comparer(name=mesh_name, 
    mask_a=radar_mask[x_lo:x_hi, y_lo:y_hi], 
    mask_b=crest_lines_to_use[x_lo:x_hi, y_lo:y_hi],
    names={},
    background=culled_background_flip[x_lo:x_hi, y_lo:y_hi],
    result_filename=f'{mesh_name}_irl={irl_radius}_r={pixel_radius}px',
    dir=folder,
    filetype='tiff',
    DPI=600
)
_ = comparer.CM(
    true_pos_color=np.asarray([0, 51, 102]), #toRGB("#2ab7ca"),
    false_pos_color=np.asarray([255, 153, 0]), #toRGB("#fed766"),
    false_neg_color=np.asarray([128, 128, 128]), #toRGB("#fe4a49"),
    true_neg_color=np.asarray([50, 50, 50]),
    background_color=toRGB("#ffffff"),
    comparison_space=(image_mask).astype(np.uint8)[x_lo:x_hi, y_lo:y_hi],
    radius=pixel_radius,
    save=True,
    figsize=(12, 12), # for comparison
)
_ = comparer.detect_b_to_a_thicken(radius=pixel_radius, DPI=600)
# _ = comparer.detect_a_to_b(radius=pixel_radius, DPI=600)
# _ = comparer.GPR_image()
# _ = comparer.GPR_lines()
# _ = comparer.crest_lines()
# _ = comparer.centerline_over_background(background=culled_background_flip[x_lo:x_hi, y_lo:y_hi].astype(np.uint8))